# Nebula Graph (CE)


## Deploy


```docker compose up```


## Observações:

* Sintaxe: nGQL/openCypher
* Comparações: https://www.nebula-graph.io/comparison_of_editions
* Não tem Built-in General Graph Algorithms
* API do Spark funciona, mas é necessário trabalhar com versões especificas (2.4, 3.0, etc). 



## Python

In [4]:
! pip install nebula3-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

# Configuração da conexão
config = Config()
config.max_connection_pool_size = 10

# Inicializar pool de conexões
connection_pool = ConnectionPool()
ok = connection_pool.init([('127.0.0.1', 9669)], config)

if not ok:
    print("Erro ao conectar com NebulaGraph")
    exit(1)


In [6]:
with connection_pool.session_context('root', 'nebula') as session:
    # Criar o espaço de grafo
    session.execute('CREATE SPACE IF NOT EXISTS exemplo_grafo(vid_type=FIXED_STRING(256))')
    
    # Aguardar a criação do espaço
    import time
    time.sleep(10)
    
    # Usar o espaço criado
    session.execute('USE exemplo_grafo')
    
    # Criar tags (tipos de vértices)
    session.execute('''
        CREATE TAG IF NOT EXISTS pessoa(
            nome string, 
            idade int, 
            profissao string
        )
    ''')
    
    session.execute('''
        CREATE TAG IF NOT EXISTS empresa(
            nome string, 
            setor string, 
            funcionarios int
        )
    ''')
    
    # Criar tipos de arestas
    session.execute('''
        CREATE EDGE IF NOT EXISTS trabalha_em(
            cargo string, 
            salario double, 
            inicio_trabalho date
        )
    ''')
    
    session.execute('''
        CREATE EDGE IF NOT EXISTS conhece(
            desde date, 
            proximidade string
        )
    ''')
    
    print("Esquema do grafo criado com sucesso!")


Esquema do grafo criado com sucesso!


In [7]:
with connection_pool.session_context('root', 'nebula') as session:
    session.execute('USE exemplo_grafo')
    
    # Inserir pessoas
    pessoas = [
        ("p001", "João Silva", 35, "Engenheiro"),
        ("p002", "Maria Santos", 29, "Designer"),
        ("p003", "Carlos Lima", 42, "Gerente"),
        ("p004", "Ana Costa", 31, "Analista")
    ]
    
    for pid, nome, idade, profissao in pessoas:
        query = f'''
        INSERT VERTEX pessoa(nome, idade, profissao) 
        VALUES "{pid}":("{nome}", {idade}, "{profissao}")
        '''
        result = session.execute(query)
        print(f"Inserido: {nome} - {result.is_succeeded()}")
    
    # Inserir empresas
    empresas = [
        ("e001", "TechCorp", "Tecnologia", 500),
        ("e002", "DesignStudio", "Design", 50),
        ("e003", "ConsultingGroup", "Consultoria", 200)
    ]
    
    for eid, nome, setor, funcionarios in empresas:
        query = f'''
        INSERT VERTEX empresa(nome, setor, funcionarios) 
        VALUES "{eid}":("{nome}", "{setor}", {funcionarios})
        '''
        result = session.execute(query)
        print(f"Inserida empresa: {nome} - {result.is_succeeded()}")


Inserido: João Silva - True
Inserido: Maria Santos - True
Inserido: Carlos Lima - True
Inserido: Ana Costa - True
Inserida empresa: TechCorp - True
Inserida empresa: DesignStudio - True
Inserida empresa: ConsultingGroup - True


In [8]:
with connection_pool.session_context('root', 'nebula') as session:
    session.execute('USE exemplo_grafo')
    
    # Relacionamentos trabalha_em
    trabalhos = [
        ("p001", "e001", "Engenheiro Senior", 8500.0, "2020-03-15"),
        ("p002", "e002", "Designer Pleno", 6000.0, "2021-07-01"),
        ("p003", "e003", "Gerente de Projeto", 12000.0, "2019-01-10"),
        ("p004", "e001", "Analista Junior", 4500.0, "2022-08-20")
    ]
    
    for pessoa_id, empresa_id, cargo, salario, inicio in trabalhos:
        query = f'''
        INSERT EDGE trabalha_em(cargo, salario, inicio_trabalho) 
        VALUES "{pessoa_id}" -> "{empresa_id}":("{cargo}", {salario}, date("{inicio}"))
        '''
        result = session.execute(query)
        print(f"Relacionamento trabalho criado: {pessoa_id} -> {empresa_id}")
    
    # Relacionamentos pessoais
    amizades = [
        ("p001", "p002", "2018-05-20", "amigo_proximo"),
        ("p001", "p004", "2022-08-20", "colega"),
        ("p002", "p003", "2019-12-10", "conhecido"),
        ("p003", "p004", "2022-09-01", "colega")
    ]
    
    for p1, p2, desde, proximidade in amizades:
        query = f'''
        INSERT EDGE conhece(desde, proximidade) 
        VALUES "{p1}" -> "{p2}":(date("{desde}"), "{proximidade}")
        '''
        result = session.execute(query)
        print(f"Relacionamento pessoal criado: {p1} -> {p2}")


Relacionamento trabalho criado: p001 -> e001
Relacionamento trabalho criado: p002 -> e002
Relacionamento trabalho criado: p003 -> e003
Relacionamento trabalho criado: p004 -> e001
Relacionamento pessoal criado: p001 -> p002
Relacionamento pessoal criado: p001 -> p004
Relacionamento pessoal criado: p002 -> p003
Relacionamento pessoal criado: p003 -> p004


In [18]:
with connection_pool.session_context('root', 'nebula') as session:
    session.execute('USE exemplo_grafo')
    
    # Consultar todas as pessoas
    result = session.execute('MATCH (p:pessoa) RETURN p.pessoa.nome, p.pessoa.idade, p.pessoa.profissao')
    print("\n=== PESSOAS NO GRAFO ===")
    for row in result:
        nome, idade, profissao = row
        print(f"Nome: {nome}, Idade: {idade}, Profissão: {profissao}")
    
    # Consultar relacionamentos de trabalho
    result = session.execute('''
        MATCH (p:pessoa)-[t:trabalha_em]->(e:empresa) 
        RETURN p.pessoa.nome, t.cargo, t.salario, e.empresa.nome
    ''')
    print("\n=== RELACIONAMENTOS DE TRABALHO ===")
    for row in result:
        pessoa, cargo, salario, empresa = row
        print(f"{pessoa} trabalha como {cargo} na {empresa} (Salário: R$ {salario})")
    
    # Consultar rede de relacionamentos pessoais
    result = session.execute('''
        MATCH (p1:pessoa)-[c:conhece]->(p2:pessoa) 
        RETURN p1.pessoa.nome, p2.pessoa.nome, c.proximidade
    ''')
    print("\n=== RELACIONAMENTOS PESSOAIS ===")
    for row in result:
        p1, p2, proximidade = row
        print(f"{p1} conhece {p2} (Relação: {proximidade})")



=== PESSOAS NO GRAFO ===
Nome: "Ana Costa", Idade: 31, Profissão: "Analista"
Nome: "Maria Santos", Idade: 29, Profissão: "Designer"
Nome: "João Silva", Idade: 35, Profissão: "Engenheiro"
Nome: "Carlos Lima", Idade: 42, Profissão: "Gerente"

=== RELACIONAMENTOS DE TRABALHO ===
"Maria Santos" trabalha como "Designer Pleno" na "DesignStudio" (Salário: R$ 6000.0)
"João Silva" trabalha como "Engenheiro Senior" na "TechCorp" (Salário: R$ 8500.0)
"Carlos Lima" trabalha como "Gerente de Projeto" na "ConsultingGroup" (Salário: R$ 12000.0)
"Ana Costa" trabalha como "Analista Junior" na "TechCorp" (Salário: R$ 4500.0)

=== RELACIONAMENTOS PESSOAIS ===
"Maria Santos" conhece "Carlos Lima" (Relação: "conhecido")
"João Silva" conhece "Maria Santos" (Relação: "amigo_proximo")
"João Silva" conhece "Ana Costa" (Relação: "colega")
"Carlos Lima" conhece "Ana Costa" (Relação: "colega")
